### This file: 
- Cleans the content for every article.
- Uses stemming on the cleaned content.
- Wordcount before and after cleaning.
- Creates a plot displaying the 10.000 most common words used in articles.
- Gives every type a broad type, so all articles are marked as either Fake or Reliable news.
- Cleans the BBC data set
- Cleans the liar data set

In [2]:
#Importing modules
import re
import pandas as pd 
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from cleantext import clean 
from collections import Counter 
from pandarallel import pandarallel
import pandas as pd

Chose the file you want to clean

If liar or bbc, scroll down until valid title. Remember to run the all_lower, clean and stem function so they can be called for the cleaning :)

In [ ]:
#Choose the wanted filename

filename = "full_train_set.csv"
# filename = "full_val_set.csv"
# filename = "full_test_set.csv"

In [3]:
df = pd.read_csv(filename)

In [4]:
temp_df = df[["content","id"]].copy()

### Wordcount before cleaning

In [ ]:
# #Raw dataframe 'content' is converted to a string.
# uncleaned_string = '\n'.join(df["content"].astype(str))

# #'uncleaned_string' is converted to a list.
# uncleaned_list=uncleaned_string.split() 

# #Converting 'uncleaned_string' to a set, so repeated element only appear once.
# uncleaned_set=set(uncleaned_list)

# #Lenght of 'uncleaned_set' is the wordcount before cleaning.
# print("Uncleaned vocalbulary:",len(uncleaned_set))

Uncleaned vocalbulary: 4970256


4970256
This is the total amounts of words in the uncleaned vocalbulary in the training set. 

The reason we have marked the above code as a comment, is because it completely eats ups ones RAM, and therefore we do not recommend running the code. We have run it when necessary and restarted the kernel, since it takes too much processessing power.

### Functions used in the cleaning process

In [9]:
#Function making text lower case
def all_lower(x):
    x= x.lower()
    return x

#Function removing special characters
def remove_special_chars(x):
    import re 
    return re.sub(r"[^a-zA-Z0-9<>]", " ", x)


#Function replacing dates with <DATE>
def sub_dates(x):
    import re 
    sub_dates = re.sub(r'\d{4}[-/]\d{2}[-/]\d{2}|\d{2}[-/]\d{2}[-/]\d{4}', ' <DATE> ', x)
    sub_dates = re.sub(r'\b[a-z]+\s\d{1,2},\s\d{4}\b', '<DATE>', sub_dates)
    sub_dates = re.sub(r'\b\d{1,2}[,]?\s[a-zA-Z]+\s\d{4}\b', '<DATE>', sub_dates)
    return sub_dates

In [7]:
#Initializing pandarallel
pandarallel.initialize(progress_bar=True) 

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


### Main cleaning function

In [5]:
def clean_df(df):
    from cleantext import clean 
    from nltk.tokenize import word_tokenize
    from nltk.corpus import stopwords

    #Stopwords from the module nltk
    stop_words = set(stopwords.words('english'))

    #Replacing NaN values with empty string.
    df['content']=df['content'].astype(str).fillna("")
    

    #Applying the 'all_lower' function on the 'content' column 
    df['content']=df['content'].parallel_apply(all_lower)

    #Applying the 'sub_dates' function on the 'content' column 
    df['content']=df['content'].parallel_apply(sub_dates)          
    
    #Applying the clean function from the 'cleantext' module on the 'content' column 
    df['content']=df['content'].parallel_apply(                   
        lambda df: clean(str(df), lower=True, no_urls=True, no_emails=True, no_numbers=True, no_punct=True, no_currency_symbols=True))
    
    #Applying word_tokenize function from nltk module, that makes 'content' to a list of strings.
    df['content']=df['content'].parallel_apply(lambda x: word_tokenize(str(x)))  
    
    #Removing the 'exclude_words' from 'content'
    exclude_words = {"number", "date", "url", "email", "<", ">"}
    
    df['content'] = df['content'].parallel_apply(
    lambda x: [word for word in x if word not in exclude_words])
    
    #Removing the 'stop_words' from 'content'
    df['content']=df['content'].parallel_apply(
        lambda x: [word for word in x if word not in stop_words])

In [8]:
#Call the clean function on the copy df
clean_df(temp_df)

### Reduction rate of vocabulary after cleaning and WITHOUT stemming.


In [ ]:
# #Dataframe 'content' is converted to a string. 'content' is now cleaned. 
# cleaned_string = '\n'.join(temp_df["content"].astype(str))

# #'cleaned_string' is converted to a list
# cleaned_list=cleaned_string.split() 

# #'cleaned_list' is converted to a set, so any repeated element only appear once.
# cleaned_set=set(cleaned_list)

# #Lenght of 'cleaned_set' is wordcount after cleaning.
# print("Cleaned vocabulary",len(cleaned_set))   

# #Wordcount before cleaning:
# prior =  4970256
 
# #Wordcount after cleaning:
# after = len(cleaned_set)  

# #Removed words in the cleaning process  
# diff = prior - after  


# print("Percentage change:", (diff / prior) * 100, "%")

Cleaned vocabulary 1908946
Percentage change: 61.59260207120116 %


1908946
This is the total amounts of words in the cleaned but NOT stemmed vocalbulary in the training set. 
Reduction Rate: 61.59260207120116 %

### Function used for stemming

In [12]:
#Importing PorterStemmer from nltk
from nltk.stem import PorterStemmer

def stemming(df):
    ps = PorterStemmer()

    #Stemming for every word in every line the 'content' column
    df['content'] = df['content'].parallel_apply(lambda x: " ".join(ps.stem(word) for word in x))


In [16]:
from nltk.stem import PorterStemmer
stemming(temp_df)

Mapping the temporary datafram back to the orignal

In [17]:
#Using map to update 'content' based on 'id'
df['content'] = df['id'].map(temp_df.set_index('id')['content'])


### Reduction rate of vocabulary after cleaning and stemming.

In [ ]:
# #Dataframe 'content' is converted to a string. 'content' is now cleaned. 
# cleaned_string = '\n'.join(temp_df["content"].astype(str))

# #'cleaned_string' is converted to a list
# cleaned_list=cleaned_string.split() 

# #'cleaned_list' is converted to a set, so any repeated element only appear once.
# cleaned_set=set(cleaned_list)

# #Lenght of 'cleaned_set' is wordcount after cleaning.
# print("Cleaned vocabulary",len(cleaned_set))   

# #Wordcount before cleaning:
# prior = 4970256 
 
# #Wordcount after cleaning:
# after = len(cleaned_set)  

# #Removed words in the cleaning process  
# diff = prior - after  

# print("Percentage change:", (diff / prior) * 100, "%")

Cleaned vocabulary 1569236
Percentage change: 68.42746128167241 %


1569236
This is the total amounts of words in the cleaned AND stemmed vocalbulary in the training set. 
Reduction Rate: 68.42746128167241 %

Depending on what file that has been cleaned, choose the right name for the saved csv

In [ ]:
#Converting the final datafram to a csv.


df.to_csv('full_train_cleaned.csv')
# df.to_csv('full_val_cleaned.csv')
# df.to_csv('full_test_cleaned.csv')



### Cleaning the BBC articles

In [ ]:
#Reading the bbc data set

bbc_filename = "BBC_broad_content.csv"
df_BBC = pd.read_csv(bbc_filename)

In [10]:
clean_df(df_BBC)

In [13]:
from nltk.stem import PorterStemmer
stemming(df_BBC)

In [ ]:
#Converting the final dataframe to a csv.
df_BBC.to_csv('bbc_cleaned.csv')

### Cleaning the liar data set

In [ ]:
#Reading the  data set
df_liar = pd.read_csv("liar_train.csv")

In [22]:
df_liar.head(1)

,ID,Label,Statement,subject(s),Speaker,Speaker job title,State info,party affiliation,barely true counts,false counts,half true counts,mostly true counts,pants on fire counts,context (venue/location),broad_category
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.1,0.2,0.3,a mailer,Fake News


In [23]:
#Creating a new column called 'content' which contain the content of the articles.
df_liar["content"] = df_liar["Statement"]

In [24]:
#Initializing pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [25]:
clean_df(df_liar)

In [26]:
from nltk.stem import PorterStemmer
stemming(df_liar)

In [27]:
#Using map to update the 'Statement' column based on ID
df_liar['Statement'] = df_liar['ID'].map(df_liar.set_index('ID')['content'])

In [28]:
#Deleting the 'content' column
df_liar.drop('content', axis=1, inplace=True)

In [30]:
df_liar.head(1)

,ID,Label,Statement,subject(s),Speaker,Speaker job title,State info,party affiliation,barely true counts,false counts,half true counts,mostly true counts,pants on fire counts,context (venue/location),broad_category
0,2635.json,false,say anni list polit group support thirdtrimest...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.1,0.2,0.3,a mailer,Fake News


In [ ]:
#Converting the final liar dataframe to a csv.
df_liar.to_csv("liar_cleaned.csv", index=False)